**This notebook is an exercise in the [Feature Engineering](https://www.kaggle.com/learn/feature-engineering) course.  You can reference the tutorial at [this link](https://www.kaggle.com/matleonard/feature-selection).**

---


# Introduction

In this exercise you'll use some feature selection algorithms to improve your model. Some methods take a while to run, so you'll write functions and verify they work on small samples.

To begin, run the code cell below to set up the exercise.

In [1]:
# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.feature_engineering.ex4 import *

Then run the following cell. It takes a minute or so to run.

In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing, metrics
import lightgbm as lgb

import os

clicks = pd.read_parquet('../input/feature-engineering-data/baseline_data.pqt')
data_files = ['count_encodings.pqt',
              'catboost_encodings.pqt',
              'interactions.pqt',
              'past_6hr_events.pqt',
              'downloads.pqt',
              'time_deltas.pqt',
              'svd_encodings.pqt']
data_root = '../input/feature-engineering-data'
for file in data_files:
    features = pd.read_parquet(os.path.join(data_root, file))
    clicks = clicks.join(features)

def get_data_splits(dataframe, valid_fraction=0.1):

    dataframe = dataframe.sort_values('click_time')
    valid_rows = int(len(dataframe) * valid_fraction)
    train = dataframe[:-valid_rows * 2]
    # valid size == test size, last two sections of the data
    valid = dataframe[-valid_rows * 2:-valid_rows]
    test = dataframe[-valid_rows:]
    
    return train, valid, test

def train_model(train, valid, test=None, feature_cols=None):
    if feature_cols is None:
        feature_cols = train.columns.drop(['click_time', 'attributed_time',
                                           'is_attributed'])
    dtrain = lgb.Dataset(train[feature_cols], label=train['is_attributed'])
    dvalid = lgb.Dataset(valid[feature_cols], label=valid['is_attributed'])
    
    param = {'num_leaves': 64, 'objective': 'binary', 
             'metric': 'auc', 'seed': 7}
    num_round = 1000
    print("Training model!")
    bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], 
                    early_stopping_rounds=20, verbose_eval=False)
    
    valid_pred = bst.predict(valid[feature_cols])
    valid_score = metrics.roc_auc_score(valid['is_attributed'], valid_pred)
    print(f"Validation AUC score: {valid_score}")
    
    if test is not None: 
        test_pred = bst.predict(test[feature_cols])
        test_score = metrics.roc_auc_score(test['is_attributed'], test_pred)
        return bst, valid_score, test_score
    else:
        return bst, valid_score

## Baseline Score

Let's look at the baseline score for all the features we've made so far.

In [3]:
train, valid, test = get_data_splits(clicks)
_, baseline_score = train_model(train, valid)

Training model!
[LightGBM] [Info] Number of positive: 363974, number of negative: 1476475
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.930623 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14325
[LightGBM] [Info] Number of data points in the train set: 1840449, number of used features: 91
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.197764 -> initscore=-1.400330
[LightGBM] [Info] Start training from score -1.400330
Validation AUC score: 0.9659876667590743


### 1) Which data to use for feature selection?

Since many feature selection methods require calculating statistics from the dataset, should you use all the data for feature selection?

Run the following line after you've decided your answer.

In [4]:
# Check your answer (Run this code cell to receive credit!)
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> Including validation and test data within the feature selection is a source of leakage. You'll want to perform feature selection on the train set only, then use the results there to remove features from the validation and test sets.

Now we have 91 features we're using for predictions. With all these features, there is a good chance the model is overfitting the data. We might be able to reduce the overfitting by removing some features. Of course, the model's performance might decrease. But at least we'd be making the model smaller and faster without losing much performance.

### 2) Univariate Feature Selection

Below, use `SelectKBest` with the `f_classif` scoring function to choose 40 features from the 91 features in the data. 

In [5]:
from sklearn.feature_selection import SelectKBest, f_classif
feature_cols = clicks.columns.drop(['click_time', 'attributed_time', 'is_attributed'])
train, valid, test = get_data_splits(clicks)

# Create the selector, keeping 40 features
selector = SelectKBest(f_classif, k=40)

# Use the selector to retrieve the best features
X_new = selector.fit_transform(train[feature_cols], train['is_attributed'])

# Get back the kept features as a DataFrame with dropped columns as all 0s
selected_features = pd.DataFrame(selector.inverse_transform(X_new),index=train.index,columns=feature_cols)

# Find the columns that were dropped
dropped_columns = selected_features.columns[selected_features.var() == 0]

# Check your answer
q_2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# Uncomment these lines if you need some guidance
# q_2.hint()
# q_2.solution()

In [7]:
_ = train_model(train.drop(dropped_columns, axis=1), 
                valid.drop(dropped_columns, axis=1))

Training model!
[LightGBM] [Info] Number of positive: 363974, number of negative: 1476475
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.424438 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6258
[LightGBM] [Info] Number of data points in the train set: 1840449, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.197764 -> initscore=-1.400330
[LightGBM] [Info] Start training from score -1.400330
Validation AUC score: 0.9625039444949599


### 3) The best value of K

With this method we can choose the best K features, but we still have to choose K ourselves. How would you find the "best" value of K? That is, you want it to be small so you're keeping the best features, but not so small that it's degrading the model's performance.

Run the following line after you've decided your answer.

In [8]:
# Check your answer (Run this code cell to receive credit!)
q_3.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> To find the best value of K, you can fit multiple models with increasing values of K, then choose the smallest K with validation score above some threshold or some other criteria. A good way to do this is loop over values of K and record the validation scores for each iteration.

### 4) Use L1 regularization for feature selection

Now try a more powerful approach using L1 regularization. Implement a function `select_features_l1` that returns a list of features to keep.

Use a `LogisticRegression` classifier model with an L1 penalty to select the features. For the model, set:
- the random state to 7,
- the regularization parameter to 0.1,
- and the solver to `'liblinear'`.

Fit the model then use `SelectFromModel` to return a model with the selected features.

The checking code will run your function on a sample from the dataset to provide more immediate feedback.

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

def select_features_l1(X, y):
    """Return selected features using logistic regression with an L1 penalty."""
    log_mol = LogisticRegression(C=0.1, penalty="l1", random_state=7, solver='liblinear').fit(X, y)
    model = SelectFromModel(log_mol, prefit=True)
    X1 = model.transform(X)
    selected_col = pd.DataFrame(model.inverse_transform(X1),index=X.index,columns=X.columns)

    ex_col = selected_col.columns[selected_col.var() != 0]

    return ex_col

# Check your answer
q_4.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [10]:
# Uncomment these if you're feeling stuck
#q_4.hint()
#q_4.solution()

In [11]:
n_samples = 10000
X, y = train[feature_cols][:n_samples], train['is_attributed'][:n_samples]
selected = select_features_l1(X, y)

dropped_columns = feature_cols.drop(selected)
_ = train_model(train.drop(dropped_columns, axis=1), 
                valid.drop(dropped_columns, axis=1))

Training model!
[LightGBM] [Info] Number of positive: 363974, number of negative: 1476475
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.066547 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4507
[LightGBM] [Info] Number of data points in the train set: 1840449, number of used features: 25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.197764 -> initscore=-1.400330
[LightGBM] [Info] Start training from score -1.400330
Validation AUC score: 0.9656558241516175


### 5) Feature Selection with Trees

Since we're using a tree-based model, using another tree-based model for feature selection might produce better results. What would you do different to select the features using a trees classifier?

Run the following line after you've decided your answer.

In [12]:
# Check your answer (Run this code cell to receive credit!)
q_5.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> You could use something like `RandomForestClassifier` or `ExtraTreesClassifier` to find feature importances. `SelectFromModel` can use the feature importances to find the best features.

### 6) Top K features with L1 regularization

Here you've set the regularization parameter `C=0.1` which led to some number of features being dropped. However, by setting `C` you aren't able to choose a certain number of features to keep. What would you do to keep the top K important features using L1 regularization?

Run the following line after you've decided your answer.

In [13]:
# Check your answer (Run this code cell to receive credit!)
q_6.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> To select a certain number of features with L1 regularization, you need to find the regularization parameter that leaves the desired number of features. To do this you can iterate over models with different regularization parameters from low to high and choose the one that leaves K features. Note that for the scikit-learn models C is the *inverse* of the regularization strength.

Congratulations on finishing this course! To keep learning, check out the rest of [our courses](https://www.kaggle.com/learn/overview). The machine learning explainability and deep learning courses are great next skills to learn!

---




*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161443) to chat with other Learners.*